<a href="https://colab.research.google.com/github/bbrauser/DS-Unit-1-Sprint-1-Data-Wrangling-and-Storytelling/blob/master/Brad_Brauser_LS_DS_214_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 4*

---

# Logistic Regression


## Assignment 🌯

You'll use a [**dataset of 400+ burrito reviews**](https://srcole.github.io/100burritos/). How accurately can you predict whether a burrito is rated 'Great'?

> We have developed a 10-dimensional system for rating the burritos in San Diego. ... Generate models for what makes a burrito great and investigate correlations in its dimensions.

- [X] Do train/validate/test split. Train on reviews from 2016 & earlier. Validate on 2017. Test on 2018 & later.
- [X] Begin with baselines for classification.
- [X] Use scikit-learn for logistic regression.
- [X] Get your model's validation accuracy. (Multiple times if you try multiple iterations.)
- [X] Get your model's test accuracy. (One time, at the end.)
- [ ] Commit your notebook to your fork of the GitHub repo.


## Stretch Goals

- [ ] Add your own stretch goal(s) !
- [ ] Make exploratory visualizations.
- [ ] Do one-hot encoding.
- [ ] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Get and plot your coefficients.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [1]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Linear-Models/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [2]:
# Load data downloaded from https://srcole.github.io/100burritos/
import pandas as pd
df = pd.read_csv(DATA_PATH+'burritos/burritos.csv')

In [3]:
# Derive binary classification target:
# We define a 'Great' burrito as having an
# overall rating of 4 or higher, on a 5 point scale.
# Drop unrated burritos.
df = df.dropna(subset=['overall'])
df['Great'] = df['overall'] >= 4

In [4]:
# Clean/combine the Burrito categories
df['Burrito'] = df['Burrito'].str.lower()

california = df['Burrito'].str.contains('california')
asada = df['Burrito'].str.contains('asada')
surf = df['Burrito'].str.contains('surf')
carnitas = df['Burrito'].str.contains('carnitas')

df.loc[california, 'Burrito'] = 'California'
df.loc[asada, 'Burrito'] = 'Asada'
df.loc[surf, 'Burrito'] = 'Surf & Turf'
df.loc[carnitas, 'Burrito'] = 'Carnitas'
df.loc[~california & ~asada & ~surf & ~carnitas, 'Burrito'] = 'Other'

In [5]:
# # Drop some high cardinality categoricals
# df = df.drop(columns=['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood'])

In [6]:
# # Drop some columns to prevent "leakage"
# df = df.drop(columns=['Rec', 'overall'])

In [28]:
# Train/validate/test split
train = df[df['Date'] <= '12/31/2016']
test = df[df['Date'] >= '1/1/2018']
validate = df[(df['Date'] <= '12/31/2017') & (df['Date'] >= '1/1/2017')]

#  Wrangle function
def wrangle(df):
  df.drop(['Notes', 'Location', 'Reviewer', 'Address', 'URL', 'Neighborhood', 
           'Rec', 'overall', 'Date', 'Chips', 'Beef',	'Pico',	'Guac', 'Cheese',
           'Fries',	'Sour cream',	'Pork', 'Chicken', 'Shrimp', 'Fish', 'Rice', 
           'Beans', 'Lettuce', 'Tomato', 'Bell peper', 'Carrots', 'Cabbage', 
           'Sauce', 'Salsa.1', 'Cilantro', 'Onion', 'Taquito', 'Pineapple',	'Ham',	
           'Chile relleno', 'Nopales', 'Lobster', 'Queso', 'Egg', 'Mushroom', 
           'Bacon', 'Sushi', 'Avocado', 'Corn', 'Zucchini', 'Unreliable', 'NonSD'], 
          axis=1, inplace=True)
  return df

X = wrangle(train).fillna(0)
y = X.pop('Great')

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [29]:
X

,Burrito,Yelp,Google,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
0,California,3.5,4.2,6.49,3.0,0.0,0.0,0.0,0.0,0.00,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0
1,California,3.5,3.3,5.45,3.5,0.0,0.0,0.0,0.0,0.00,2.0,3.5,2.5,2.5,2.0,4.0,3.5,2.5,5.0
2,Carnitas,0.0,0.0,4.85,1.5,0.0,0.0,0.0,0.0,0.00,3.0,2.0,2.5,3.0,4.5,4.0,3.0,3.0,5.0
3,Asada,0.0,0.0,5.25,2.0,0.0,0.0,0.0,0.0,0.00,3.0,2.0,3.5,3.0,4.0,5.0,4.0,4.0,5.0
4,California,4.0,3.8,6.59,4.0,0.0,0.0,0.0,0.0,0.00,4.0,5.0,4.0,3.5,4.5,5.0,2.5,4.5,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,California,0.0,0.0,7.25,4.0,0.0,0.0,0.0,0.0,0.00,4.0,5.0,4.0,5.0,5.0,3.0,3.0,4.0,5.0
387,Other,4.5,3.8,4.19,3.0,0.0,0.0,0.0,0.0,0.00,3.0,5.0,2.0,2.0,4.0,1.0,4.0,3.0,4.0
388,California,3.5,4.3,7.00,5.0,0.0,0.0,0.0,0.0,0.00,5.0,5.0,5.0,5.0,5.0,5.0,4.0,5.0,5.0
389,Other,0.0,0.0,8.50,4.0,0.0,0.0,21.0,23.5,0.92,4.0,4.0,3.0,3.5,1.0,2.0,3.0,3.0,1.0


In [30]:
# Train/validation split
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

print(X.shape)
print(X_train.shape)
print(X_val.shape)

assert X.shape[0] == X_train.shape[0] + X_val.shape[0]
assert X_train.shape[0] == len(y_train)
assert X_val.shape[0] == len(y_val)

(84, 19)
(67, 19)
(17, 19)


In [31]:
# Baseline
y_train.value_counts(normalize=True)

False    0.61194
True     0.38806
Name: Great, dtype: float64

Need to beat a 0.61

In [32]:
# Importing classes
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from category_encoders import OneHotEncoder 
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [33]:
# Instantiating Pipeline
log_model = Pipeline([('ohe', OneHotEncoder(cols=['Burrito'])), 
                      ('imputer', SimpleImputer()), ('classifier', LogisticRegression())])

In [34]:
log_model

Pipeline(memory=None,
         steps=[('ohe',
                 OneHotEncoder(cols=['Burrito'], drop_invariant=False,
                               handle_missing='value', handle_unknown='value',
                               return_df=True, use_cat_names=False,
                               verbose=0)),
                ('imputer',
                 SimpleImputer(add_indicator=False, copy=True, fill_value=None,
                               missing_values=nan, strategy='mean',
                               verbose=0)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_intercept=True, intercept_scaling=1,
                                    l1_ratio=None, max_iter=100,
                                    multi_class='auto', n_jobs=None,
                                    penalty='l2', random_state=None,
                                    solver='lbfgs', tol=0.0001, verbose=0,
            

In [35]:
# Fitting model to training data
log_model.fit(X_train, y_train);

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [36]:
X_val.head()

,Burrito,Yelp,Google,Cost,Hunger,Mass (g),Density (g/mL),Length,Circum,Volume,Tortilla,Temp,Meat,Fillings,Meat:filling,Uniformity,Salsa,Synergy,Wrap
311,Other,0.0,0.0,7.90,2.0,0.0,0.0,19.5,24.0,0.89,4.2,4.0,4.2,3.8,4.0,3.0,3.8,4.5,5.0
0,California,3.5,4.2,6.49,3.0,0.0,0.0,0.0,0.0,0.00,3.0,5.0,3.0,3.5,4.0,4.0,4.0,4.0,4.0
296,California,4.0,4.3,5.65,3.0,0.0,0.0,19.5,22.0,0.75,4.0,1.5,2.0,3.0,4.2,4.0,3.0,2.0,4.5
260,Other,0.0,0.0,4.99,4.0,0.0,0.0,0.0,0.0,0.00,4.5,5.0,3.0,3.0,3.0,2.5,0.0,2.5,5.0
250,California,4.0,3.8,6.10,4.0,0.0,0.0,22.0,22.0,0.85,3.5,3.5,2.0,3.5,4.5,3.5,3.0,3.5,4.0


In [37]:
print('Training Accuracy:', log_model.score(X_train, y_train))
print('Validation Accuracy:', log_model.score(X_val, y_val))

Training Accuracy: 0.9552238805970149
Validation Accuracy: 0.7058823529411765


In [ ]:
# X_train['Guac'].str.replace('x', '1', case = False).fillna('0').astype(int)

In [ ]:
# X_train['Guac'].str.contains('x|X', regex = True).head()

In [ ]:
X_train[:,2:8] = X_train.applymap(lambda n:1 if n else 0)
X_train.head()